In [1]:
import numpy as np
from openai import AsyncOpenAI
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from getpass import getpass
import json
import traceback
import time
import asyncio
from dotenv import load_dotenv

load_dotenv()

False

In [2]:
# Check if OPENAI_API_KEY environment variable is set
if "OPENAI_API_KEY" not in os.environ:
    print("Enter your OpenAI API key:")
    os.environ["OPENAI_API_KEY"] = getpass()

# evaluator setup
# openai.api_key = os.environ.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_base = os.environ.get("OPENAI_API_BASE")
evaluator_model = "gpt-4-1106-preview"

Enter your OpenAI API key:


In [3]:
llm = OpenAI(api_key=OPENAI_API_KEY)
# llm = AsyncOpenAI(api_key=OPENAI_API_KEY)
# llm = OpenAI()

In [4]:
# get question
question_ind = np.random.choice(60)
question_data = json.load(open("/Users/vasu/Desktop/NLP /project/chatbot_ic_NLP/lib/backend/evaluations/evaluation_ques.json", "r"))[question_ind]
user_question = question_data["question"]
print(user_question)

How do sedentary living and a high-sugar diet contribute to type 2 diabetes development?


In [5]:
# imports
import warnings

# Suppress TypedStorage deprecation warning
warnings.filterwarnings("ignore", category=UserWarning, message="TypedStorage is deprecated.*")
warnings.filterwarnings("ignore", category=UserWarning, message="LangChainDeprecationWarning: The function `run` was deprecated.*")


import chromadb
from chromadb.utils import embedding_functions

import json
import chromadb
from chromadb.config import Settings
from chromadb import EmbeddingFunction
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


/Users/vasu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def get_context_with_max_similarity(user_question):
    # Initialize SentenceTransformer model
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L12-v2", normalize_embeddings=True)

    # Embed the user's question
    user_question_embedding = sentence_transformer_ef([user_question])[0]

    # Perform the query using Chroma
    search_results = collection_whole.query(query_embeddings=[user_question_embedding], n_results=5)

    # Find the index of the context with the maximum similarity score
    max_similarity_index = search_results['distances'][0].index(max(search_results['distances'][0]))

    # Get the context with the maximum similarity score
    context_with_max_similarity = search_results['documents'][0][max_similarity_index]

    return context_with_max_similarity

chroma_internet_client = chromadb.HttpClient(host='16.171.68.145', port=8000, settings=Settings(allow_reset=True))

collection_whole = chroma_internet_client.get_collection('pubmed_whole')

In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

QUERY_PROMPT_MODEL = PromptTemplate(
    input_variables=["context"],
    template="""
You are an AI language model assistant. Your task is to generate answer
    by taking information from the relevant context provided from a vector 
    database. By considering multiple perspectives on the user question, your goal is to help
    the user understand the concept of the question asked that is also relevant to the context provided. 
    Provide these answers with proper type setting.

    Original question: {question}
    Context : {context}
""",)

MULTI_QUERY_PROMPT_MODEL = PromptTemplate(
    input_variables=["context"],
    template="""You are an AI language model assistant. Your task is to generate questions based on the provided context. 
    Generate list of 3 questions that help to explore different aspects of the context and deepen the understanding of the topic. 
    Provide these questions with proper type setting.

    
    Context : {context}
""",)


# Initialize the LLMChain

llm_chain_multi_query= LLMChain(prompt=MULTI_QUERY_PROMPT_MODEL, llm=llm)
llm_chain_answer = LLMChain(prompt=QUERY_PROMPT_MODEL, llm=llm)

def generate_questions(context):
    # Generate questions based on the provided context
    generated_questions = llm_chain_multi_query.run({'context': context})
    
    return generated_questions



context_with_max_similarity = get_context_with_max_similarity(user_question)

generated_questions = generate_questions(context_with_max_similarity)


#FETCHING 3 questions

# Split the generated questions string into a list of individual questions
generated_questions_list = generated_questions.split('\n')

# Filter out any empty strings from the list
generated_questions_list = [question.strip() for question in generated_questions_list if question.strip()]

# Extract the first three questions from the list
question1 = generated_questions_list[0] if len(generated_questions_list) > 0 else ""
question2 = generated_questions_list[1] if len(generated_questions_list) > 1 else ""
question3 = generated_questions_list[2] if len(generated_questions_list) > 2 else ""



q1_with_max_simialrity=get_context_with_max_similarity(question1)
q2_with_max_simialrity=get_context_with_max_similarity(question2)
q3_with_max_simialrity=get_context_with_max_similarity(question3)


new_context=context_with_max_similarity+q1_with_max_simialrity+q2_with_max_simialrity+q3_with_max_simialrity

print("Question: ",user_question)
answer = llm_chain_answer.run({'context': new_context, 'question': user_question})
print("Answer: ",answer)

/Users/vasu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Question:  How do sedentary living and a high-sugar diet contribute to type 2 diabetes development?
Answer:  
There are several factors that can contribute to the development of type 2 diabetes, including sedentary living and a high-sugar diet. Sedentary living refers to a lifestyle that involves little to no physical activity, while a high-sugar diet consists of consuming excessive amounts of sugary foods and beverages.

These lifestyle choices can lead to an increase in blood glucose levels, which can eventually lead to insulin resistance and the development of type 2 diabetes. Insulin resistance occurs when the body's cells become less responsive to insulin, the hormone responsible for regulating blood sugar levels. This can result in elevated blood sugar levels and, over time, can lead to a diagnosis of type 2 diabetes.

Additionally, sedentary living and a high-sugar diet can also contribute to other health issues, such as cardiovascular disease and obesity, which are risk factors

In [8]:

import openai
QUERY_PROMPT = f"""
You are an AI language model assistant. Your task is to generate answer
    by taking information from the relevant context provided from a vector 
    database. By considering multiple perspectives on the user question, your goal is to help
    the user understand the concept of the question asked that is also relevant to the context provided. 
    Provide these answers with proper type setting.

    Original question: {user_question}
    Context : {new_context}
"""



In [9]:


print("Question: ",user_question)
print("-------------------------")
for i in range(10):
    try:
        chat_completion = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": QUERY_PROMPT}],
            stream=False,
        )
        if isinstance(chat_completion, tuple):
            # Handle tuple response
            print("Received tuple response:", chat_completion)
            # Optionally, retry or handle the tuple response as needed
            continue

        if isinstance(chat_completion, dict):
            # Not stream
            answer = chat_completion.choices[0].message.content
        else:
            
            answer = chat_completion.choices[0].message.content
        break
    except Exception as exc:
        print(traceback.format_exc())
        print(exc)
        if i != 9:
            print(f"Retrying... (i = {i})")
            time.sleep(3)

print(answer)

Question:  How do sedentary living and a high-sugar diet contribute to type 2 diabetes development?
-------------------------
Sedentary living and a high-sugar diet can contribute to the development of type 2 diabetes through various mechanisms. 

1. **Sedentary living:** 
   - Lack of physical activity can lead to weight gain and obesity, which are major risk factors for type 2 diabetes. 
   - Sedentary behavior can also affect insulin sensitivity, making it harder for the body to regulate blood sugar levels effectively.
   - Regular exercise helps to maintain a healthy weight, improve insulin sensitivity, and reduce the risk of developing type 2 diabetes.

2. **High-sugar diet:** 
   - Consuming excessive amounts of sugar can lead to weight gain, insulin resistance, and inflammation in the body, all of which are linked to the development of type 2 diabetes.
   - High sugar intake can also contribute to the development of metabolic syndrome, a cluster of conditions that increase the r

In [10]:
eval_prompt = f"""
        Your task is to evaluate a student's response to a given exercise. In the exercise, the student is provided with some general context consisting of 3 titles and abstracts of medical articles.
        The student is furthermore asked a question, which he should answer correctly making use of the provided context.
        The exercise tests the student's abilities regarding grammar, reading comprehension and logical reasoning. The student's answer starts after the *** symbol.
        Please provide your general assessment about the answer provided by the student (the part after the *** symbol).
        Is it correct? Is it grammatically correct? Is it consistent with the given context?
        Furthermore, grade the student’s answer in terms of grammar, coherence, consistency with the context and whether it is correct or not. Moreover, please provide your best guess of what the academic degree of the student might be, as reflected from the answer. Choose from possible 4 possible categories: A: no degree. B: bachelor's degree. C: master's degree. D: doctoral degree. Use the following grade format: Grammar: #/10, Coherence: #/10, Context: #/10, Correctness: #/10, where the "#" should be replaces by a number between 0 (worst) and 10 (best).
        Context: {new_context}
        Question: {user_question}
        Answer: *** {answer}
"""

In [11]:
for i in range(10):
    try:
        chat_completion = openai.chat.completions.create(
            model=evaluator_model,
            messages=[{"role": "user", "content": QUERY_PROMPT}],
            stream=False,
        )
        if isinstance(chat_completion, tuple):
            # Handle tuple response
            print("Received tuple response:", chat_completion)
            # Optionally, retry or handle the tuple response as needed
            continue

        if isinstance(chat_completion, dict):
            # Not stream
            answer = chat_completion.choices[0].message.content
        else:
            
            answer = chat_completion.choices[0].message.content
        break
    except Exception as exc:
        print(traceback.format_exc())
        print(exc)
        if i != 9:
            print(f"Retrying... (i = {i})")
            time.sleep(3)

print(answer)


### Understanding How Sedentary Living and High-Sugar Diet Contribute to Type 2 Diabetes

Type 2 diabetes is a multifaceted metabolic disorder characterized by elevated blood glucose levels over a prolonged period. The development of this condition is influenced by various genetic, lifestyle, and environmental factors. The provided context from scientific literature highlights several aspects of glucose metabolism and the role of monitoring in the context of diabetes management and risk assessment. To address the original question in light of the given context, let's consider the relevant concepts:

#### Sedentary Living:
- **Impact on Glucose Trajectory:** Sedentary behavior contributes to a less favorable glucose trajectory, which relates to progressive deterioration in glucose control over time, as indicated in the work by Arthur L M Swislocki. This trajectory can potentially be monitored using mathematical modeling and might be a useful predictor of type 2 diabetes development.
- *